# Single Label Training and Multiple Label Prediction 

## PART B.

### Includes NNs on Multi label learning

1.Plain FNN

2.CNN 

Stacked MLP 

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import data_utils

Train_x, Train_y, Test_x, Test_y = data_utils.load_data()


Train_x = np.asarray(Train_x)
Train_y = np.asarray(Train_y)
Test_x  = np.asarray(Test_x)
Test_y  = np.asarray(Test_y)


Using TensorFlow backend.


In [2]:
## Training and Testing Input

X_train_single = Train_x[200:1700]

X_test_multi = Train_x[:200]
## Training and Testing Output

from sklearn.preprocessing import MultiLabelBinarizer

y = np.load('../behav_Exp/pri_sec_emotion.npy')

from keras.utils import to_categorical

y_train = Train_y[200:1700]
y_train = to_categorical(y_train)


mlb = MultiLabelBinarizer()
y_test = mlb.fit_transform(y)

## Train: X_train_single;y_train   
## Test: X_test_multi;y_test

In [3]:
print X_train_single.shape
print y_train.shape

(1500, 2304)
(1500, 7)


In [64]:
### MLP structure 

#### Dense > Dropout > Dense > Dropout > Dense(Output)

from keras.models import Sequential
from keras.layers import Dense,Dropout

MLP = Sequential()

MLP.add(Dense(400, input_shape= (2304,), activation=  'relu'))

MLP.add(Dropout(0.2))

MLP.add(Dense(200, activation='relu'))

MLP.add(Dropout(0.2))

MLP.add(Dense((7), activation= 'sigmoid'))

In [65]:
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
MLP.compile(optimizer=Adam(),  loss = binary_crossentropy )

In [66]:
MLP.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 400)               922000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               80200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 1407      
Total params: 1,003,607
Trainable params: 1,003,607
Non-trainable params: 0
_________________________________________________________________


In [67]:
MLP.fit(X_train_single, y_train, epochs=50, batch_size=50,verbose=0)

In [68]:
pred = MLP.predict_proba(X_test_multi)

 32/200 [===>..........................] - ETA: 0s

In [69]:
pred.shape

(200, 7)

In [70]:
pred[1]

array([  6.24468157e-06,   5.35409818e-05,   1.74935703e-04,
         2.64674109e-05,   1.32020846e-01,   3.10235051e-03,
         7.20095456e-01], dtype=float32)

## How to Output Multiple Label Predictions

### Let a == pred_proba matrix, then argmax(1) to -1, argmax(-2) to 1; then argmin(1) to 1,
### Final Let a[!=1] = 0 

In [71]:
a = pred
a[np.arange(len(a)),a.argmax(1)] = -1 

a[np.arange(len(a)),a.argmax(1)] = 1

a[np.arange(len(a)),a.argmin(1)] = 1

a[a != 1] = 0

In [72]:
a.astype('int')

array([[0, 0, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 1, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0]])

In [73]:
from sklearn.metrics import *

In [74]:
print 'Hamming Loss:', hamming_loss(y_test, a)
print 'Convergence Error:', coverage_error(y_test, a)
print 'Label Ranking loss:', label_ranking_loss(y_test, a)
print 'F1-scores(Weighted)', f1_score(y_test, a, average='weighted')
print 'F1-scores(Micro)', f1_score(y_test, a, average='micro')
print 'F1-scores(Macro)', f1_score(y_test, a, average='macro')
print 'F1-scores(Samples)', f1_score(y_test, a, average='samples')
print 'Average Precision', average_precision_score(y_test,a)


Hamming Loss: 0.402857142857
Convergence Error: 6.7
Label Ranking loss: 0.761333333333
F1-scores(Weighted) 0.257998193704
F1-scores(Micro) 0.282442748092
F1-scores(Macro) 0.262093473979
F1-scores(Samples) 0.280833333333
Average Precision 0.376450229263


## CNN

In [82]:
# tune the input size 

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


img_cols,img_rows = 48, 48

if K.image_data_format() == 'channels_first':
    X_train = X_train_single.reshape(X_train_single.shape[0], 1, img_rows, img_cols)
    X_test = X_test_multi.reshape(X_test_multi.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train_single.reshape(X_train_single.shape[0], img_rows, img_cols, 1)
    X_test = X_test_multi.reshape(X_test_multi.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [83]:
print X_test.shape

(200, 48, 48, 1)


In [84]:
cnn = Sequential()
cnn.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.25))
cnn.add(Dense(7, activation='sigmoid'))

In [85]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 30976)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               3965056   
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
__________

In [86]:
from keras.optimizers import Adam
from keras.losses import binary_crossentropy

cnn.compile(optimizer=Adam(),  loss = binary_crossentropy )

In [87]:
cnn.fit(X_train, y_train, epochs=50, batch_size=50,verbose=0)

In [88]:
pred = cnn.predict_proba(X_test)

192/200 [===========================>..] - ETA: 0s

In [89]:
a = pred
a[np.arange(len(a)),a.argmax(1)] = -1 

a[np.arange(len(a)),a.argmax(1)] = 1

a[np.arange(len(a)),a.argmin(1)] = 1

a[a != 1] = 0

In [90]:
a[1]

array([ 0.,  0.,  0.,  0.,  1.,  0.,  1.], dtype=float32)

In [91]:
a.astype('int')

array([[1, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0]])

In [92]:
from sklearn.metrics import *
print 'Hamming Loss:', hamming_loss(y_test, a)
print 'Convergence Error:', coverage_error(y_test, a)
print 'Label Ranking loss:', label_ranking_loss(y_test, a)
print 'F1-scores(Weighted)', f1_score(y_test, a, average='weighted')
print 'F1-scores(Micro)', f1_score(y_test, a, average='micro')
print 'F1-scores(Macro)', f1_score(y_test, a, average='macro')
print 'F1-scores(Samples)', f1_score(y_test, a, average='samples')
print 'Average Precision', average_precision_score(y_test,a)

Hamming Loss: 0.387142857143
Convergence Error: 6.6
Label Ranking loss: 0.733
F1-scores(Weighted) 0.286547626759
F1-scores(Micro) 0.310432569975
F1-scores(Macro) 0.281124493241
F1-scores(Samples) 0.31
Average Precision 0.384007353427
